# Imports

In [1]:
from music_downloader import MusicDownloader
from utils import Credentials

from joblib import Parallel, delayed
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import os

# Global variables

In [2]:
DRIVER_PATH = './chromedriver'
SCOPE = "user-read-recently-played"
SPOTIFY_DOWNLOADER_WEBSITE =  'https://www.soundloaders.com/spotify-downloader/'
OUTPUT_DIR_NAME = 'mouka'


playlist_urls = [
#     'https://open.spotify.com/playlist/5PQ03g9b9jFvrzE7OLAHPH?si=c6db43bad93d4308',
#     'https://open.spotify.com/playlist/7CtI3MSMDCoHtCwpQ5rWCO?si=29a6a148f2d24496',
#     'https://open.spotify.com/playlist/1RanMy39MDxrA1inDDP59G?si=a694e5e277e940da',
#     'https://open.spotify.com/playlist/6gRwrLeRS9FKRjQWYtPf37?si=d42308d8144f41a6',
#     'https://open.spotify.com/playlist/1RanMy39MDxrA1inDDP59G?si=3670450c33194cdb',
#     'https://open.spotify.com/playlist/6gRwrLeRS9FKRjQWYtPf37?si=2718862df0834708',
#     'https://open.spotify.com/playlist/5VSWx8qQm78pmHyK7f2qxt?si=71254f3e1a194a48',
    'https://open.spotify.com/playlist/37i9dQZF1E8MuDBVfgStke?si=0c871867034e4e49',
    'https://open.spotify.com/playlist/4I9IzJCAUYeZknkImPLhtE?si=8e2576a8c0c04a89'
]

# Main

In [3]:
cred = Credentials()

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36")
WINDOW_SIZE = "920,800"
# options.add_argument("--headless")
options.add_argument("--windo>w-size=%s" % WINDOW_SIZE)
options.add_argument("--mute-audio")

dir = os.path.join(os.getcwd(), OUTPUT_DIR_NAME)
prefs = {
            "download.default_directory" : OUTPUT_DIR_NAME,
            "savefile.default_directory": OUTPUT_DIR_NAME,}

options.add_experimental_option("prefs", prefs)
options.add_experimental_option('excludeSwitches', ['enable-logging'])
downloader = MusicDownloader(
        output_dir_name=OUTPUT_DIR_NAME, 
        cred=cred, 
        scope=SCOPE,
        spotify_downloader_website=SPOTIFY_DOWNLOADER_WEBSITE,
        driver_path=DRIVER_PATH
        )

downloader.options = options
downloader.dir = dir

In [4]:
artists = []
names = []
urls = []
    
for playlist_url in playlist_urls:
    
    results = downloader.sp.playlist(playlist_url)
    playlist = results['tracks']['items']

    
    for song in playlist:
        track = song['track']
        artist = track['artists'][0]['name']
        name = track['name']
        url = track['external_urls']['spotify']

        artists.append(artist)
        names.append(name)
        urls.append(url)
        print(artist, " – ", name, " – ", url)

Oliver & Tom  –  Bruges  –  https://open.spotify.com/track/5n4o4LOhLKnK4GVPtMhCAe
Weird Sounding Dude  –  B84 - Mixed  –  https://open.spotify.com/track/0ttK3GxbsnMOsU1M8QzuRq
Dowden  –  Coil  –  https://open.spotify.com/track/4yVHeZHpuOzgAe5HGPhYsf
Felix Raphael  –  Miracle - M.O.S. Remix  –  https://open.spotify.com/track/619LXLCGcsQ7dLC09gZGQe
Juan Deminicis  –  Cycles  –  https://open.spotify.com/track/4cCUH25SukXQAXgqiAQ6an
Jiminy Hop  –  Colatrix - Matter Remix  –  https://open.spotify.com/track/5bbVxxLdaxWqfigHKbKO5y
Oliver & Tom  –  Valencia  –  https://open.spotify.com/track/4FsducmxCSJRkN78soOkan
Greg Ochman  –  Hili  –  https://open.spotify.com/track/6jvHIvyewyEPfPBGw2ttYO
RIGOONI  –  The Day I Reframed The Moon  –  https://open.spotify.com/track/2FUlyAfiqDoQD1Q8djZhCW
Eric Lune  –  Embers - Ric Niels Remix  –  https://open.spotify.com/track/3o4mXX73TR8MjOVgdDmaRG
Alex O'Rion  –  The Serpent - Sebastian Sellares Remix  –  https://open.spotify.com/track/3Yoy2TC4y9ePEKBT8ZZkJA

In [5]:
summary = pd.DataFrame(
            {
                "name": names,
                "artist": artists,
                "url": urls,
                })

summary = summary.drop_duplicates()

In [6]:
summary

,name,artist,url
0,Bruges,Oliver & Tom,https://open.spotify.com/track/5n4o4LOhLKnK4GV...
1,B84 - Mixed,Weird Sounding Dude,https://open.spotify.com/track/0ttK3GxbsnMOsU1...
2,Coil,Dowden,https://open.spotify.com/track/4yVHeZHpuOzgAe5...
3,Miracle - M.O.S. Remix,Felix Raphael,https://open.spotify.com/track/619LXLCGcsQ7dLC...
4,Cycles,Juan Deminicis,https://open.spotify.com/track/4cCUH25SukXQAXg...
...,...,...,...
104,Golden Bell,Gadi Mitrani,https://open.spotify.com/track/4D9v2CLP7igKClc...
105,Ry,Christian Löffler,https://open.spotify.com/track/01hyJrAHoH26GHS...
106,A Blue Rose For Daniele,RIGOONI,https://open.spotify.com/track/7mwTCrvDiyC9SUb...
107,Real Or Nature - Booka Shade Remix,Verboten Berlin,https://open.spotify.com/track/7dWLMy4fAxmvE1G...


In [ ]:
Parallel(n_jobs=10)(delayed(downloader.download_song)(url) 
                        for url in summary['url'] )


In [15]:
import time
url = summary['url'].iloc[0]
print(url)
driver = downloader._connect_driver()
driver.get(downloader.spotify_downloader_website)
time.sleep(2)
search_bar = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[1]/form/input')
search_bar.clear()
search_bar.send_keys(url)
search_bar.send_keys(Keys.ENTER)
time.sleep(2)
download_button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[1]/div[3]/div/div[2]/button')
download_button.click()
time.sleep(3)

download_window = '//*[@id="__next"]/main/div/div[1]/div[5]/div/div'
try:
    element = WebDriverWait(driver, 5*60).until_not( EC.presence_of_element_located((By.XPATH, download_window)))
    time.sleep(60)
finally:
    driver.quit()


